In [ ]:
from flask import Flask, request, render_template_string, jsonify
import os
from google import genai
# from dotenv import load_dotenv

# load_dotenv()

app = Flask(__name__)
client = genai.Client()

# 首頁：chatbot 介面
@app.route("/")
def index():
    return render_template_string('''
    <!DOCTYPE html>
    <html lang="zh-TW">
    <head>
        <meta charset="UTF-8">
        <title>Gemini Chatbot</title>
        <style>
            body { font-family: Arial, sans-serif; margin: 40px; }
            #response { margin-top: 20px; border: 1px solid #ccc; padding: 10px; min-height: 60px; }
            button { margin-right: 10px; }
        </style>
    </head>
    <body>
        <h2>Gemini Chatbot</h2>
        <input type="text" id="user_input" placeholder="請輸入訊息" style="width:300px;">
        <button onclick="startChat()">開始</button>
        <button onclick="clearAll()">清除</button>
        <div id="response"></div>
        <script>
            function startChat() {
                var user_input = document.getElementById('user_input').value;
                if (!user_input) return;
                document.getElementById('response').innerHTML = '處理中...';
                fetch('/chat', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ question: user_input })
                })
                .then(res => res.json())
                .then(data => {
                    document.getElementById('response').innerHTML = data.reply;
                })
                .catch(() => {
                    document.getElementById('response').innerHTML = '發生錯誤';
                });
            }
            function clearAll() {
                document.getElementById('user_input').value = '';
                document.getElementById('response').innerHTML = '';
            }
        </script>
    </body>
    </html>
    ''')

# 處理 Gemini API
@app.route("/chat", methods=["POST"])
def chat():
    data = request.get_json()
    question = data.get('question', '')
    if not question:
        return jsonify({'reply': '請輸入問題'}), 400
    try:
        response = client.models.generate_content(
            model="gemini-2.5-flash", contents=question
        )
        reply = response.text.replace("```html","").replace("```","")
    except Exception as e:
        reply = f"API 錯誤: {e}"
    return jsonify({'reply': reply})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True)